In [ ]:
from __future__ import print_function, division
import numpy as np
import scipy as sp

import mat_neuron._model as mat
from dstrf import strf, mle

# plotting packages
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

In [ ]:
# model parameters: (α1, α2, ω, τ1, τ2, tref)
matparams = np.asarray([100, 2, 7, 10, 200, 2], dtype='d')
model_dt = 0.5

# data parameters
duration = 100000
n_samples = int(duration / model_dt)
n_assim = 1
n_test = 0

# assimilation parameters
nthreads = 4
nwalkers = 500
nsteps = 500
matparams_i = [0,1,2]
matparams_n = len(matparams_i)
matparams_f = matparams[matparams_i]

In [ ]:
# convolution - simple alpha kernel
from scipy.signal import resample
stim_dt = 10.0
upsample = int(stim_dt / model_dt)

# alpha filter
tau_max = 600
tau_h = 50
tt = np.arange(0, tau_max, stim_dt)
ka = np.flipud(tt / tau_h * np.exp(-tt / tau_h))

# difference of gammas:
from scipy.special import gamma
tau_h1 = 600. / 32
tau_h2 = 600. / 16
kg1 = 1/(gamma(6)*tau_h1)*(tt/tau_h1)**5 * np.exp(-tt/tau_h1)
kg2 = 1/(gamma(6)*tau_h2)*(tt/tau_h2)**5 * np.exp(-tt/tau_h2)
kg = np.flipud(kg1 - kg2 / 1.5)
kg /= np.linalg.norm(kg)

# select a filter
k1 = (ka * 7)[::-1]
plt.plot(k1)
    

In [ ]:
def filter_stimulus(S, k1):
    return np.convolve(S, k1, mode="full")[:S.size]


def predict_spikes(V, params, dt, upsample):
    a1, a2, omega, t1, t2, tref = params
    return mat.predict_poisson(V - omega, (a1, a2), (t1, t2), tref, 
                               dt, upsample)


In [ ]:
# generate data to fit
np.random.seed(1)
mat.random_seed(1)
data = []
stim = np.random.randn(int(n_samples / (stim_dt / model_dt)))
stim[:100] = 0
        
V = filter_stimulus(stim, k1)
for i in range(n_assim + n_test):
    spikes = predict_spikes(V, matparams, model_dt, upsample)
    H = mat.adaptation(spikes, matparams[3:5], model_dt)
    z = np.nonzero(spikes)[0]
    d = {"H": H,
         "duration": duration,
         "spike_t": z, 
         "spike_v": spikes,
         "loglike": mat.log_likelihood_poisson(V - matparams[2], H, spikes, matparams[:2], model_dt, upsample)
        }
    data.append(d)

# split into assimilation and test sets
assim_data = data[:n_assim]
test_data = data[n_assim:]

In [ ]:
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
for i, d in enumerate(data):
    ax2.vlines(d["spike_t"], i, i + 0.5)
ax1.plot(V)
ax1.set_xlim(0, 8000 / upsample)
ax2.set_xlim(0, 8000)
print("log likelihood: {}".format(np.sum([d["loglike"] for d in data])))
print("spikes: {}; rate: {} / dt".format(np.mean([d["spike_t"].size for d in data]), 
                                         np.mean([d["spike_t"].size / d["duration"] for d in data])))

In [ ]:
# initial guess of parameters using ML
spikes = np.stack([d["spike_v"] for d in data], axis=1)
w0 = mle.estimate(stim, spikes, k1.size, matparams[3:5], 
                        stim_dt, model_dt)

In [ ]:
mparam_ml = w0[[1, 2, 0]]
rf_ml = w0[:2:-1]
print(mparam_ml)
plt.plot(rf_ml)
plt.plot(k1)

In [ ]:
# estimate parameters using emcee
from neurofit import priors, costs, utils, startpos
import emcee

mat_prior = priors.joint_independent(
                [ priors.uniform(-50,  200),
                  priors.uniform(-5,   10),
                  priors.uniform( 0,  20),
                ])
startparams = np.asarray([[-50, 200],
                          [-5, 10],
                          [0, 20],
                         ], dtype='d')

# lasso prior on RF parameters
rf_lambda = 1.0

# this is the local copy of the parameters that we'll update in each step
mparams = matparams.copy()
rfparams = k1.copy()

def lnpost_dyn(theta):
    """Posterior probability for dynamical parameters"""
    mparams[matparams_i] = theta[:matparams_n]
    rfparams[:] = theta[matparams_n:]
    rf_prior = -np.sum(np.abs(rfparams)) * rf_lambda
    ll = mat_prior(theta[:matparams_n]) + rf_prior
    if not np.isfinite(ll):
        return -np.inf
    lp = 0
    V = filter_stimulus(stim, rfparams)
    for d in assim_data:
        lp += mat.log_likelihood_poisson(V - matparams[2], d["H"], d["spike_v"], matparams[:2], model_dt, upsample)
    return ll + lp        

In [ ]:
# theoretically this is as good as it can get
theta_true = np.concatenate([matparams_f, k1])
print("lnpost of p_true: {}".format(lnpost_dyn(theta_true)))
# initial state is a gaussian ball around the ML estimate
theta_ml = np.concatenate([mparam_ml, rf_ml])
p0 = startpos.normal_independent(nwalkers, theta_ml, np.abs(theta_ml) * 0.2)
theta_0 = np.median(p0, 0)
print("lnpost of p0 median: {}".format(lnpost_dyn(theta_0)))
%timeit lnpost_dyn(theta_true)

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, theta_true.size, lnpost_dyn, threads=nthreads)
tracker = utils.convergence_tracker(nsteps, 25)

for pos, prob, _ in tracker(sampler.sample(p0, iterations=nsteps, storechain=False)): 
    continue

In [ ]:
print("lnpost of p median: {}".format(np.median(prob)))
print("average acceptance fraction: {}".format(sampler.acceptance_fraction.mean()))
theta = np.median(pos, 0)
mparams[matparams_i] = theta[:matparams_n]
rfparams[:] = theta[matparams_n:]
plt.plot(k1)
plt.plot(rfparams)
print(matparams_f)
print(theta[:matparams_n])

In [ ]:
from corner import corner
sns.set_style("whitegrid")

mpos = pos[:,:matparams_n]
matlabs = ['a1','a2','w']
c = corner(mpos,
           #range=[sp for sp in startparams],
       bins=50, smooth=2,smooth1d=0,
       labels=matlabs,
       truths=matparams_f)

In [ ]:
# see how well predictions line up
d = assim_data[0]
Vpred = filter_stimulus(stim, rfparams)

plt.plot(V[:400])
plt.plot(Vpred[:400])

In [ ]:
for i, d in enumerate(data):
    plt.vlines(d["spike_t"], i, i + 0.5, 'r')

for i in range(len(data), len(data) + 10):
    S = predict_spikes(Vpred, mparams, model_dt, upsample)
    spk_t = S.nonzero()[0]
    plt.vlines(spk_t, i, i + 0.5)

plt.xlim(0, 10000)